In [1]:
import json
import numpy as np
import pandas as pd
import plotly.express as px
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

---
### Load the dataset and split train test split
---

In [2]:
# Load the data
with open('../data/curated-data-train.json') as f:
    data_train = json.load(f)
df_train = pd.DataFrame(data_train).T
# Filter dataframe to only include the columns we want
df_train = df_train[['back', 'left', 'right']]
# Separate the three data into the same label
for i in range(3):
    for pose in ['back', 'left', 'right']:
        df_train[f'{pose}_{i}'] = df_train[pose].apply(lambda x: x[i])
# Drop the original columns
df_train = df_train.drop(columns=['back', 'left', 'right'])
# Expand the data into a single column
df_train = df_train.melt()
# Rename the 'variable' values to only include the pose
df_train['variable'] = df_train['variable'].apply(lambda x: x.split('_')[0])
# Rename the columns
df_train.columns = ['posture', 'reading']
df_train.head()

,posture,reading
0,back,"[[23.64, 23.91, 23.95, 23.82, 24.64, 24.24, 25..."
1,back,"[[24.09, 24.33, 24.21, 24.42, 25.24, 24.97, 25..."
2,back,"[[24.92, 24.97, 25.39, 25.1, 26.22, 26.13, 26...."
3,back,"[[26.39, 25.85, 26.58, 26.84, 27.47, 27.24, 27..."
4,back,"[[25.03, 24.98, 24.68, 24.7, 25.71, 25.73, 25...."


In [3]:
# Load the data
with open('../data/curated-data-test.json') as f:
    data_test = json.load(f)
df_test = pd.DataFrame(data_test).T
# Filter dataframe to only include the columns we want
df_test = df_test[['back', 'left', 'right']]
# Separate the three data into the same label
for i in range(3):
    for pose in ['back', 'left', 'right']:
        df_test[f'{pose}_{i}'] = df_test[pose].apply(lambda x: x[i])
# Drop the original columns
df_test = df_test.drop(columns=['back', 'left', 'right'])
# Expand the data into a single column
df_test = df_test.melt()
# Rename the 'variable' values to only include the pose
df_test['variable'] = df_test['variable'].apply(lambda x: x.split('_')[0])
# Rename the columns
df_test.columns = ['posture', 'reading']
df_test.head()

,posture,reading
0,back,"[[24.34, 24.45, 24.23, 24.19, 24.62, 24.79, 25..."
1,back,"[[23.11, 23.47, 23.46, 23.26, 24.27, 24.1, 24...."
2,back,"[[23.27, 23.49, 23.71, 23.37, 24.37, 24.4, 25...."
3,back,"[[24.16, 23.52, 24.36, 23.67, 24.85, 24.1, 24...."
4,back,"[[23.32, 22.9, 23.3, 22.98, 23.96, 23.38, 24.0..."


In [162]:
# Split the data into training and testing sets
X_train = df_train['reading']
y_train = df_train['posture']
X_test = df_test['reading']
y_test = df_test['posture']
# One-hot encode the labels
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values

In [163]:
X_train = np.array([np.array(x) for x in X_train])
X_train = np.expand_dims(X_train, -1)
X_test = np.array([np.array(x) for x in X_test])
X_test = np.expand_dims(X_test, -1)
y_train = np.array([np.array(x) for x in y_train])
y_test = np.array([np.array(x) for x in y_test])
# Convert the data to tensors
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

---
### Traininig a generic CNN model
---

In [164]:
def create_model():
    model = keras.Sequential(
        [
            layers.Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same', input_shape=X_train.shape[1:]),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(4, kernel_size=(3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),  # Flatten the output of the CNN
            layers.Dense(32, activation='relu'),
            layers.Dense(16, activation='relu'),
            layers.Dense(3, activation='softmax'),
        ]
    )
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'],
    )
    return model

In [165]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=20,
    min_delta=0.001,
    restore_best_weights=True,
)
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    patience=5
)

In [172]:
X_train[[1]]

<tf.Tensor: shape=(24, 32, 1), dtype=float64, numpy=
array([[[24.09],
        [24.33],
        [24.21],
        [24.42],
        [25.24],
        [24.97],
        [25.65],
        [25.33],
        [26.08],
        [26.12],
        [26.48],
        [26.82],
        [27.93],
        [28.26],
        [28.35],
        [28.4 ],
        [27.9 ],
        [27.68],
        [26.48],
        [26.22],
        [25.85],
        [25.48],
        [25.28],
        [24.97],
        [24.98],
        [25.02],
        [24.5 ],
        [24.26],
        [24.16],
        [23.41],
        [22.41],
        [21.56]],

       [[24.14],
        [24.69],
        [24.46],
        [24.92],
        [25.15],
        [25.52],
        [25.39],
        [25.45],
        [26.25],
        [26.54],
        [26.72],
        [27.47],
        [28.82],
        [29.5 ],
        [29.74],
        [29.75],
        [28.78],
        [28.61],
        [26.93],
        [26.64],
        [25.87],
        [25.66],
        [25.43],
        [2

In [174]:
# Does some cross-validation
kf = KFold(n_splits=5, shuffle=True)
scores = []
for train_index, test_index in kf.split(X_train):
    X_train_cv, X_test_cv = tf.gather(X_train, train_index), tf.gather(X_train, test_index)
    y_train_cv, y_test_cv = tf.gather(y_train, train_index), tf.gather(y_train, test_index)
    model = create_model()
    model.fit(
        X_train_cv,
        y_train_cv,
        validation_data=(X_test_cv, y_test_cv),
        epochs=1000,
        batch_size=16,
        callbacks=[early_stopping, reduce_lr],
    )
    score = model.evaluate(X_test_cv, y_test_cv)
    scores.append(score[1])
print(f'Average accuracy: {np.mean(scores)}')

Epoch 1/1000
7/7 [==============================] - 3s 53ms/step - loss: 12.7788 - accuracy: 0.2870 - val_loss: 9.2598 - val_accuracy: 0.2222 - lr: 0.0010
Epoch 2/1000
7/7 [==============================] - 0s 19ms/step - loss: 4.7675 - accuracy: 0.3148 - val_loss: 1.4130 - val_accuracy: 0.4815 - lr: 0.0010
Epoch 3/1000
7/7 [==============================] - 0s 16ms/step - loss: 2.3350 - accuracy: 0.3611 - val_loss: 1.0909 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 4/1000
7/7 [==============================] - 0s 14ms/step - loss: 1.4617 - accuracy: 0.3704 - val_loss: 1.1615 - val_accuracy: 0.3704 - lr: 0.0010
Epoch 5/1000
7/7 [==============================] - 0s 13ms/step - loss: 1.2495 - accuracy: 0.5185 - val_loss: 0.9980 - val_accuracy: 0.4074 - lr: 0.0010
Epoch 6/1000
7/7 [==============================] - 0s 14ms/step - loss: 0.9965 - accuracy: 0.5463 - val_loss: 1.1384 - val_accuracy: 0.4074 - lr: 0.0010
Epoch 7/1000
7/7 [==============================] - 0s 14ms/step - loss: 0.

In [ ]:
px.line(hist.history, y=['accuracy', 'val_accuracy'], labels={'index': 'epoch', 'value': 'accuracy'})

In [ ]:
preds = model.predict(X_test)
preds = np.argmax(preds, axis=1)
y_test_max = np.argmax(y_test, axis=1)
# Create a confusion matrix
confusion_matrix = tf.math.confusion_matrix(labels=y_test_max, predictions=preds).numpy()
# Create a dataframe from the confusion matrix
confusion_matrix = pd.DataFrame(confusion_matrix, index=['back', 'left', 'right'], columns=['back', 'left', 'right'])
# Plot the confusion matrix
px.imshow(confusion_matrix, color_continuous_scale='Blues')

2/2 [==============================] - 0s 8ms/step


In [ ]:
values = {
    0: 'back',
    1: 'left',
    2: 'right',
}

# Get the X inputs that were misclassified
misclassified = X_test[y_test_max != preds]
# Get the y inputs that were misclassified
real_labels = y_test_max[y_test_max != preds]
pred_labels = preds[y_test_max != preds]
# Plot the misclassified inputs
for i in range(len(misclassified)):
    fig = px.imshow(
        img = misclassified[i, :, :, 0]
    )
    fig.update_layout(
        title=f'Predicted: {values[pred_labels[i]]}, '
        f'Actual: {values[real_labels[i]]}'
    )
    fig.show()

---
### Predicting for whole data
---

In [ ]:
X_total = np.array([np.array(x) for x in X])
X_total = np.expand_dims(X_total, -1)
X_total = tf.convert_to_tensor(X_total)
y_total = tf.convert_to_tensor(y)

In [ ]:
y_pred = model.predict(X_total)
y_pred_max = np.argmax(y_pred, axis=1)
y_total_max = np.argmax(y_total, axis=1)

6/6 [==============================] - 0s 6ms/step


In [ ]:
# Create a confusion matrix
confusion_matrix = tf.math.confusion_matrix(labels=y_total_max, predictions=y_pred_max).numpy()
# Create a dataframe from the confusion matrix
confusion_matrix = pd.DataFrame(confusion_matrix, index=['back', 'left', 'right'], columns=['back', 'left', 'right'])
# Plot the confusion matrix
fig = px.imshow(confusion_matrix, color_continuous_scale='Blues')
fig.update_layout(
    title='Confusion Matrix for CNN',
    xaxis_title='Predicted',
    yaxis_title='Actual',
)

In [ ]:
print(f'Accuracy: {np.sum(y_pred_max == y_total_max) / len(y_total_max)}')

Accuracy: 0.8465608465608465


In [ ]:
model.save_weights('./checkpoints/my_checkpoint')